# Lending Club Risk Prediction

The aim of this project is to build a machine learning model that will allow us to accurately predict if a borrower will pay off their loan on time or not. 

The dataset we'll be using is the complete loan data for all [Lending Club](https://www.lendingclub.com) loans issued from 2012 to 2013, which can be downloaded [here](https://www.lendingclub.com/info/download-data.action).

A description of each column can be found [here](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097).

# Importing and Cleaning the Data

In [1]:
import pandas as pd
import numpy as np

# We'll skip the first row because it provides information about the dataset's origin rather than column titles.
loans_2013 = pd.read_csv('loans2013.csv', skiprows=1)

og_cols = loans_2013.shape[1]
# Drop columns in which at least half of the column data is missing
missing_info_thresh = loans_2013.shape[0]/2
loans_2013.dropna(thresh=missing_info_thresh, axis=1, inplace=True)

# Save processed dataset incase we want to explore the original later
loans_2013.to_csv('processed_loans2013.csv', index=False)

print('{} columns dropped'.format(og_cols - loans_2013.shape[1]))

loans_2013.head(1)

/home/zrupp/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,47,123,124,125,128,129,130,133) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


58 columns dropped


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,12000.0,12000.0,12000.0,36 months,7.62%,373.94,A,A3,Systems Engineer,3 years,...,100.0,0.0,0.0,233004.0,46738.0,14800.0,53404.0,N,Cash,N


That's a lot of columns. Determining which columns to keep is going to take some time. Let's familiarize ourselves with the columns and determine which columns:

- Leak information from the future
- Don't affect a borrower's ability to pay back a loan
- Need to be cleaned because of poor formatting
- Require more data or too much processing to be usefully turned into a feature
- Contain redundant information

We'll want to import the data dictionary to make this process a little less painful.

# Columns 1-20

In [2]:
# We'll need to expand the maximum column width to view the descriptions
pd.options.display.max_colwidth = 250

# Import the data dictionary to easily reference what each column describes
data_dict = pd.read_excel('LCDataDictionary.xlsx', sheet_name='LoanStats')

data_dict.head()

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.


In [3]:
def explain_columns(start, end=None):
    '''
    Returns a dataframe with index, column name, description of column, and first value in column
    
    start: starting index
    end: ending index 
    '''

    first_n_cols = loans_2013.columns[start:end]
    indices = []
    first_n_vals = pd.Series()


    for col in first_n_cols:
        index = list(data_dict[data_dict['LoanStatNew'] == col].index)
        indices.extend(index)
        first_n_vals = first_n_vals.append(pd.Series(loans_2013[col][0], index))

    first_n = data_dict[data_dict.index.isin(indices)]
    
    

    return pd.concat([first_n, pd.DataFrame(data=first_n_vals, columns=['first value'])], axis=1).reset_index(drop=True)


explain_columns(0, 20)

,LoanStatNew,Description,first value
0,addr_state,The state provided by the borrower in the loan application,TX
1,annual_inc,The self-reported annual income provided by the borrower during registration.,96500
2,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.,3 years
3,emp_title,The job title supplied by the Borrower when applying for the loan.*,Systems Engineer
4,funded_amnt,The total amount committed to that loan at that point in time.,12000
5,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.,12000
6,grade,LC assigned loan grade,A
7,home_ownership,"The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER",MORTGAGE
8,installment,The monthly payment owed by the borrower if the loan originates.,373.94
9,int_rate,Interest Rate on the loan,7.62%


In [4]:
# It may be meaningless, but let's find out what that asterisk signifies
data_dict.iloc[-1:]

,LoanStatNew,Description
152,NaN,* Employer Title replaces Employer Name for all loans listed after 9/23/2013


Columns to drop:

- emp_title: relevance is suspect. Also likely contains a ton of different possible categorical values which means turning this into a useful feature will take too much processing to be useful
- funded_amnt: leaks future data
- funded_amnt_inv: leaks future data
- grade: basically a weaker version of int_rate
- issue_d: leaks future data
- sub_grade: same as grade
- zip_code: basically the same as addr_state

In [5]:
dropped_cols = ['emp_title', 'funded_amnt', 'funded_amnt_inv', 'grade', 'issue_d', 'sub_grade', 'zip_code']
len(dropped_cols)

7

# Columns 21-40

In [6]:
explain_columns(20, 40)

,LoanStatNew,Description,first value
0,collection_recovery_fee,post charge off collection fee,0
1,delinq_2yrs,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years,0
2,dti,"A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.",12.61
3,earliest_cr_line,The month the borrower's earliest reported credit line was opened,Sep-2003
4,initial_list_status,"The initial listing status of the loan. Possible values are – W, F",f
5,inq_last_6mths,The number of inquiries in past 6 months (excluding auto and mortgage inquiries),0
6,last_pymnt_d,Last month payment was received,Jun-2016
7,open_acc,The number of open credit lines in the borrower's credit file.,17
8,out_prncp,Remaining outstanding principal for total amount funded,0
9,out_prncp_inv,Remaining outstanding principal for portion of total amount funded by investors,0


Columns to drop:
- collection_recovery_fee: leaks future data
- last_pymnt_d: leaks future data
- out_prncp: leaks future data
- out_prncp_inv: same as out_prncp
- recoveries: leaks future data
- total_pymnt: leaks future data
- total_pymnt_inv: same as total_pymnt
- total_rec_int: leaks future data
- total_rec_late_fee: leaks future data
- total_rec_prncp: leaks future data


In [7]:
dropped_cols.extend([
    'collection_recovery_fee', 
    'last_pymnt_d',
    'out_prncp',
    'out_prncp_inv',
    'recoveries',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_int',
    'total_rec_late_fee',
    'total_rec_prncp'
])

len(dropped_cols)

17

# Columns 41-60

In [8]:
explain_columns(40, 60)

,LoanStatNew,Description,first value
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.,0
1,acc_open_past_24mths,Number of trades opened in past 24 months.,4
2,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers,Individual
3,avg_cur_bal,Average current balance of all accounts,11783
4,bc_open_to_buy,Total open to buy on revolving bankcards.,2441
5,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.,83.5
6,chargeoff_within_12_mths,Number of charge-offs within 12 months,0
7,collections_12_mths_ex_med,Number of collections in 12 months excluding medical collections,0
8,delinq_amnt,The past-due amount owed for the accounts on which the borrower is now delinquent.,0
9,last_credit_pull_d,The most recent month LC pulled credit for this loan,Mar-2019


Columns to drop:
- last_pymnt_amnt: leaks future data (after loan start date)


In [9]:
dropped_cols.extend([
    'last_pymnt_amnt',
])

len(dropped_cols)

18

# Columns 61-80

In [10]:
explain_columns(60,80)

,LoanStatNew,Description,first value
0,mths_since_recent_bc,Months since most recent bankcard account opened.,10.0
1,mths_since_recent_inq,Months since most recent inquiry.,10.0
2,num_accts_ever_120_pd,Number of accounts ever 120 or more days past due,0.0
3,num_actv_bc_tl,Number of currently active bankcard accounts,4.0
4,num_actv_rev_tl,Number of currently active revolving trades,5.0
5,num_bc_sats,Number of satisfactory bankcard accounts,4.0
6,num_bc_tl,Number of bankcard accounts,10.0
7,num_il_tl,Number of installment accounts,15.0
8,num_op_rev_tl,Number of open revolving accounts,8.0
9,num_rev_accts,Number of revolving accounts,14.0


None of these columns seem problematic.

# Remaining Columns

In [11]:
explain_columns(80)

,LoanStatNew,Description,first value
0,tot_hi_cred_lim,Total high credit/credit limit,233004
1,total_bal_ex_mort,Total credit balance excluding mortgage,46738
2,total_bc_limit,Total bankcard high credit/credit limit,14800
3,total_il_high_credit_limit,Total installment high credit/credit limit,53404
4,hardship_flag,Flags whether or not the borrower is on a hardship plan,N
5,disbursement_method,"The method by which the borrower receives their loan. Possible values are: CASH, DIRECT_PAY",Cash
6,debt_settlement_flag,"Flags whether or not the borrower, who has charged-off, is working with a debt-settlement company.",N


These all look fine. On reflection, there are two columns I'm not entirely sure about: total_il_high_credit_limit, and num_il_tl. Because Lending Club seems to only offer installment accounts, it's possible both of these columns leak future data. We'll leave them for now, but we may want to try dropping these columns if we notice overfitting.

In [12]:
loans_2013.drop(dropped_cols, axis=1, inplace=True)
loans_2013.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,12000.0,36 months,7.62%,373.94,3 years,MORTGAGE,96500.0,Not Verified,Fully Paid,n,...,100.0,0.0,0.0,233004.0,46738.0,14800.0,53404.0,N,Cash,N
1,27050.0,36 months,10.99%,885.46,10+ years,OWN,55000.0,Verified,Fully Paid,n,...,25.0,0.0,0.0,138554.0,70186.0,35700.0,33054.0,N,Cash,N
2,12000.0,36 months,11.99%,398.52,10+ years,MORTGAGE,130000.0,Source Verified,Fully Paid,n,...,1.0,0.0,0.0,365874.0,44327.0,10700.0,57674.0,N,Cash,N
3,28000.0,36 months,7.62%,872.52,5 years,MORTGAGE,325000.0,Source Verified,Fully Paid,n,...,16.7,0.0,0.0,850886.0,199739.0,42200.0,196686.0,N,Cash,N
4,12000.0,36 months,10.99%,392.81,4 years,RENT,60000.0,Not Verified,Fully Paid,n,...,0.0,0.0,0.0,29700.0,7137.0,18100.0,0.0,N,Cash,N


69 columns may not seem like much, but I want to reduce dimensionality as much as possible before I start transforming the data, so let's see if we can drop some more columns with high null counts.

# Dropping Columns with High Null Counts

In [13]:
null_counts = loans_2013.isnull().sum()

null_counts[null_counts/loans_2013.shape[0] > .05]/loans_2013.shape[0]

tot_coll_amt                  0.147426
tot_cur_bal                   0.147426
total_rev_hi_lim              0.147426
avg_cur_bal                   0.147458
mo_sin_old_il_acct            0.180006
mo_sin_old_rev_tl_op          0.147431
mo_sin_rcnt_rev_tl_op         0.147431
mo_sin_rcnt_tl                0.147426
mths_since_recent_inq         0.148101
num_accts_ever_120_pd         0.147426
num_actv_bc_tl                0.147426
num_actv_rev_tl               0.147426
num_bc_sats                   0.085327
num_bc_tl                     0.147426
num_il_tl                     0.147426
num_op_rev_tl                 0.147426
num_rev_accts                 0.147426
num_rev_tl_bal_gt_0           0.147426
num_sats                      0.085327
num_tl_120dpd_2m              0.148786
num_tl_30dpd                  0.147426
num_tl_90g_dpd_24m            0.147426
num_tl_op_past_12m            0.147426
pct_tl_nvr_dlq                0.148239
tot_hi_cred_lim               0.147426
total_il_high_credit_limi

A significant number of columns are missing approximately 15% of the data. We could handle this a few ways:

1. We could just drop the columns entirely
2. We could drop the rows with the missing data
3. We could replace missing values with the most common value of the relevant columns.
4. We could replace missing values with 0s

I'm inclined to drop the columns entirely, but looking at these columns it seems this information may in fact be valuable. I don't want to drop the rows because that's a substantial amount of potentially meaningful data we will lose. Replacing missing values in these columns with the most common values or 0s seems like a huge mess because we're inserting data which may be extremely removed from reality, which could potentially throw off our model. It seems the best course of action would be to drop the columns entirely.